# CHAPTER 1: ACQUIRING DATA

In order to perform a pRF experiment you will need to acquire the following MRI scans:

__Echo-planar imaging (EPI)__
- One or more runs of a visual mapping stimulus
    - In this example, we use 2 runs of geometric bars in the following configuration:
        1. Bars traversing the visual field in four different directions
        2. The same bars but sweeping in a different order

- One run of sparse visual stimulation for estimating hemodynamic response functions (HRF)

- B0 field maps for your EPI scans

__Structural images__
- A high-resolution T1-weighted anatomical image with the full 32-channel head-coil on.
- (optional) if acquiring EPI scans with the top part of the head-coil off, a short T1-weighted anatomical image also with the top coil section off. Generally, when reusing an older anatomical scan for your participant, it is advisable to acquire a fast structural to aid coregistration with the old scan on which your FreeSurfer reconstruction is based.

# CHAPTER 2: DIRECTORY STRUCTURE

For each of your subjects, you will need to create an organised directory structure to perform the analysis. See here:

```
freesurfer
    |-subjects
        |-subject name
            |-label
            |-mri
            |-prf
            |-spm
            |-surf
```

The analysis for each subject is contained within a directory specific to that subject. Within the subject directory you will find:

- _label_ contains the surface labels are saved, including the maps you draw
- _mri_ contains the volume information and the original T1-weighted image reconstructed prf is the population receptive field analysis directory
- _spm_ contains the files for the pre-processing steps will reside
- _surf_ contains the surface reconstruction information from Freesurfer


__If you are using the example data__ [(downloadable here)](https://t.co/AHshMtpkyH), then you will only have two folders to start with: the prf folder, and the surf folder. This is because the structural pre-processing (`recon-all`) has already been done. You will still need to create the spm folder.

Note that most of your directory structure will be created when you run recon-all in Freesurfer (step 3), except the folders spm and prf. These will need to be created manually.

If you wish to carry out more than one pRF experiment in a single subject (e.g. two conditions), then a sensible step would be to create two folders (`prf_1` and `prf_2`) instead of one, and treat them independently from chapter 4 onwards.

Note that the subjects folder shown here is inside the Freesurfer installation. However, you can use any arbitrary folder as your subjects folder, e.g. when you have different projects and you don’t want the data to be mixed.

The best way to do this is to add this to your `.bash_profile`, this way the subject dir is changed anytime you start a shell.

You can do this by running the following line (after changing the path to where your subjects lie!):

In [103]:
!echo "export SUBJECTS_DIR="$PWD"/subjects" >> ~/.bash_profile

You can change the path to wherever you want to keep your data. However, you have to make sure that various folders FreeSurfer wants (from the original freesurfer/subjects folder) are present.

For the purpose of this notebook, we will assume that you have the example data downloaded and that your folders are structured like this:

```
freesurfer
    |- subjects
        |- example
            |- mri
            |- prf
            |- surf
            |- spm
```

# CHAPTER 3: PREPROCESSING

### 1. fMRI pre-processing

Copy your data to SUBJECT/spm and perform your standard pre-processing for fMRI images in your software of choice.
In the case of data acquired at the FIL, we recommend the following steps in SPM8:

- DICOM import\*
- Bias correction (only when using 32-channel head coil)
- Remove dummies
- Optional: Create B0 field maps
- Realign & unwarp (with or without B0 field maps)
- Optional: Slice-timing correction (Karl will tell you it’s useless)
- Co-registration
- Optional: Smoothing (pRF estimates will be poorer)
- 4D merge\*

An expanded explanation of these steps is provided in Appendix A.

From your pre-processing, you should end up with a single 4D file for each stimulation run in NiFTI format. In this example, we have 3 files: 1st run of fMRI (bars_1), 2nd run of fMRI (bars_2) and the HRF scan.

You also need to convert the T1-weighted structural from .hdr/.img to .nii.

---
\* Note that these conversion steps are unnecessary, if in your centre you can output .nii format directly when exporting the data from the scanner (e.g. at BUCNI).

### 2. Run `recon-all` (structural preprocessing)

Locate the T1-weighted file and run the automatic reconstruction:

    recon-all –i /path/to/T1.nii -s SUBJECT -all

Note SUBJECT is going to be the subject name in the freesurfer/subjects directory.

Additionally you can use the `-label_v1` option in this command to derive an anatomical prediction of V1 (based on Hinds et al., 2008, NeuroImage). This does not work on all versions of FreeSurfer (there is a bug in some versions newer than 5.0.0).

### 3. Inspect surface

Check that the reconstruction worked in each hemisphere with tksurfer. You can do this now:

In [ ]:
%%capture
!tksurfer example_2 lh inflated

In [ ]:
%%capture
!tksurfer bert rh inflated

### 4. Create occipital labels

There are two ways to create the occipital ROI:

1. Create it automatically in MatLab using MakeOccRoi command (in SamSrf/Utils)

2. Create it manually in tksurfer. Open the surface file as in step 3 and draw a wide region encompassing the occipital lobe.

Save label with meaningful names, e.g. `lh_occ.label` and `rh_occ.label`. Save these files into the subject's prf directory.

__If you are using the example data__, then this has already been done for you, but if you want to try the function, run:

In [ ]:
subject_dir = fullfile('~', 'Documents', 'prf-data', 'freesurfer', 'subjects');
MakeOccRoi('inflated', fullfile(subject_dir, 'bert', 'surf'))

If you want to check that a label has been created, you can run the following line:

In [41]:
!tksurfer -annot rh_occ.label starling44_2 rh inflated

subject is starling44_2
hemi    is rh
surface is inflated
surfer: current subjects dir: /Users/jan/Documents/prf-data/freesurfer/subjects
surfer: not in "scripts" dir ==> using cwd for session root
surfer: session root data dir ($session) set to:
surfer:     /Users/jan/Dropbox/Documents/side-projects/ninotebooks/receptive-fields/samsrf
checking for nofix files in 'inflated'
Reading image info (/Users/jan/Documents/prf-data/freesurfer/subjects/starling44_2)
Reading /Users/jan/Documents/prf-data/freesurfer/subjects/starling44_2/mri/orig.mgz
surfer: Reading header info from /Users/jan/Documents/prf-data/freesurfer/subjects/starling44_2/mri/orig.mgz
surfer: vertices=161869, faces=323734
could not read annot file /Users/jan/Documents/prf-data/freesurfer/subjects/starling44_2/surf/../label/rh.rh_occ.label.annot
No such file or directory

ERROR: could not load rh_occ.label

surfer: single buffered window
surfer: tkoInitWindow(starling44_2)
setting percentile thresholds (-1.00, 0.00, 0.00)
sur

### 5. Set up default parameters

SamSrf usually starts with a set of default parameters. These are most likely not right for your needs. You can define your own default parameters. You will need to create four char variables in MATLAB containing the default values you want. They must be saved as a file called SamSrf_defaults.mat somewhere on your MATLAB path, for example inside the SamSrf folder. Let's do so now. First, specify the path to your installation of SamSrf:

In [52]:
samsrfpath = '/path/to/Samsrf/installation';
% (in my case, it's ~/Documents/MATLAB/toolbox/SamSrf_5.7)

Then specify the default variables that are required.

First, the cortical sample steps for surface projection (see step 10):

In [43]:
def_ctxsteps = '0.5';

Then the maximal eccentricity of your stimulus (in degrees of visual angle):

In [44]:
def_eccen = '9';

The subfolder where your surface meshes reside (see step 10):

In [45]:
def_surf = '../surf';

The TR of your functional data acquisition in seconds:

In [46]:
def_tr = '2.55';

Whether to turn on/off the graphic waitbars (‘on’ or ‘off’):

In [47]:
def_wb = 'on';

And an optional parameter matrix to defined camera starting position rows defined azimuth, elevation & zoom of camera columns are for left & right hemisphere camera (we are not going to define this right now but feel free!

In [48]:
% def_views = 3 x 2 matrix

Then, we save all of these variables in a `.mat` file on your Samsrf path:

In [53]:
save(fullfile(samsrfpath, 'SamSrf_defaults.mat'), 'def_*');

# CHAPTER 4: SAMSRF PROCESSING

_Note from ninotebook team: The SamSrf steps can be done either through the GUI, or through the command line. Because the notebook format lends itself to the command line and because we think command line control is more useful because you can then run future analyses programmatically._

The first step we will take is provide the path to the subject folder we are working with. If you placed the subject's folder into this directory, then use the current directory:

In [92]:
subjects_dir = fullfile('/Users', 'jan', 'Documents', 'prf-data', 'freesurfer', 'subjects');
subject = 'starling44_2';

### 1. Check structural registration

With this function, we check that freesurfer provided us with a good registration of our brain data. To do so, we provide the `label` directory, which contains the necessary labels, to SamSrf's check registration function:

In [93]:
samsrf_checkreg(fullfile(subjects_dir, subject, 'label'));

File /Users/jan/Documents/MATLAB/toolbox/SamSrf_v5.7/samsrf_checkreg.m, line 52, in samsrf_checkreg
NII file in mri folder is ambiguous!


After a moment of loading the files a SPM check registration window will open A red line indicating the grey-white matter boundary will appear.

Look at the match-up between the anatomy and the red overlay. It should define a boundary between the white and grey matter that follows the sulcal pattern of the cortex, like in this example:

<img src="img/registration-example.png" width="500">

There are various reasons why the link between FreeSurfer’s coordinate system and SPM may fail. For example, if you fiddled with the affine transformation of the structural scan you used for FreeSurfer reconstruction (or if you resliced it or used it as source image in coregistration), then the alignment will be broken. This tool is also helpful for inspecting the surface reconstruction has worked properly (instead of step 4 above). If the red surface doesn’t cover the whole cortex but parts of the cortex are outside of it, then you will have to fix the reconstruction. This may be due to too sharply defined structural images. Subtly smoothing (~1mm kernel) the structural in SPM can help in this situation.

#### What to do if the registration is bad

If the registration is bad you can follow these quick extra steps to bring them in line:

In the terminal, go to the freesurfer/subjects/SUBJECT/mri (or wherever the nifti of your T1 is being kept) and run:

    tkregister2 --mov NAME_OF_T1.nii --s SUBJECT --regheader --noedit --reg register.dat

This will produce an output with several affine transformation matrices (4x4 matrix). You will need to copy and paste two of these into a text file called `Coregistration.txt` inside your `freesurfer/subjects/SUBJECT/surf` folder. The two matrices are the one entitled `Tmov` (2nd from top) and `RegMat` (5th). _(Note that copy&paste is annoying in the Linux terminal in that it only works via right-clicking and using the context menu – keyboard combos don’t work)_. You will need to place the `RegMat` matrix underneath the `Tmov` matrix so that in the file there will be a matrix with 8 rows and 4 columns. Finally, you must remove the semicolons at the end of the lines.

Now, if you run the check registration tool again it should automatically locate this coregistration info and use this to correctly align FreeSurfer space to native brain space.

### 2. Convert surface files

• Open a terminal window

• Change directory (cd) into the ‘Utils’ folder of the SamSrf toolbox

• Run the following command

fs_bin2asc.sh SUBJECT

This creates ASCII files of the grey-white boundary surfaces with the associated area and curvature values for each vertex, for each hemisphere in the subject inputted. The script also accepts multiple inputs to perform the ASCII conversion for more than one subject in a single command, e.g.

fs_bin2asc.sh SUBJECT1 SUBJECT2 SUBJECT3

Alternatively, this can also be done manually by following these steps:

• Open terminal

• Navigate to freesurfer/subjects/SUBJECT/surf

• Run the following commands:

    mris_convert –c lh.curv lh.white lh.curv.asc
    mris_convert –c rh.curv rh.white rh.curv.asc
    mris_convert –c lh.area lh.white lh.area.asc
    mris_convert –c rh.area rh.white rh.area.asc
    mris_convert –c lh.thickness lh.white lh.thickness.asc
    mris_convert –c rh.thickness rh.white rh.thickness.asc

### 3. Project your functional volume data onto the surface

In this step, we get the functional data ready for fitting the pRF model. This involves warping the data onto the surface so that we have surface functional data to which we can then fit the model.

We use the SamSrf function `samsrf_vol2srf()` for this.

We are going to do this for both hemispheres, so let's make list of them:

In [73]:
hemispheres = {'lh', 'rh'};

Let's also make a list of the functional images we have:

In [94]:
functional_images = dir(fullfile(subjects_dir, subject, 'afni', 'MC', '*ts2anat.nii'));
functional_images = {functional_images(:).name}';
disp(functional_images);

'pRF_21_ts2anat.nii'
    'pRF_23_ts2anat.nii'
    'pRF_25_ts2anat.nii'
    'pRF_27_ts2anat.nii'


Now we loop over each of the functional images, and the hemispheres, to project each one individually.

In [96]:
for i = 1:numel(functional_images)
    for j = 1:numel(hemispheres)
        
        
        functional_image = fullfile(subjects_dir, subject, 'afni', 'MC', functional_images{i});
        hemisphere = hemispheres{j};
        structural_image = fullfile(subjects_dir, subject, 'mri', 'orig.nii');
        hemisphere_surface = fullfile(subjects_dir, subject, 'surf', hemisphere);
        
        ctxsteps = 0.5;
        rule = '~';
        nrmls = 1;
        
        samsrf_vol2srf(functional_image, structural_image,...
                        hemisphere_surface, ctxsteps, rule, nrmls);
    end
end

Using mean of steps through cortex.
Saved lh_pRF_21_ts2anat.mat.
 
Using mean of steps through cortex.
Saved rh_pRF_21_ts2anat.mat.
 
Using mean of steps through cortex.
Saved lh_pRF_23_ts2anat.mat.
 
Using mean of steps through cortex.
Saved rh_pRF_23_ts2anat.mat.
 
Using mean of steps through cortex.
Saved lh_pRF_25_ts2anat.mat.
 
Using mean of steps through cortex.
Saved rh_pRF_25_ts2anat.mat.
 
Using mean of steps through cortex.
Saved lh_pRF_27_ts2anat.mat.
 
Using mean of steps through cortex.
Saved rh_pRF_27_ts2anat.mat.


This will take a few minutes (or longer if you have many cortical sampling steps). Once it's done, SamSrf will output all the files you gave it in .mat format with the prefix 'lh_' or 'rh_' appended to indicate which hemisphere it belongs to. Repeat for the other hemisphere.

_Remember to do this for both the functional runs from pRF mapping as well as the HRF measurement run (if you have one). The functions used to extract the HRF also work in surface space so they need this data._

### 9. Create apertures

Running the following function will open a file selector window. In that window, 

Generate
Choose the results file generated when you ran the stimulation in the scanner

In [97]:
Create_Apertures;